In [3]:
#GOOGLE SCHOLAR SEARCH OF KEY WORDS
from bs4 import BeautifulSoup
import pandas as pd
import requests

url= 'https://scholar.google.com/scholar'

#set headers
headers = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) \
    AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36' 
    #english
}

params = {
    #english
    'hl': 'en',

    'start': 0,
    'q': 'circular economy waste household' # maybe add 'trash' 'garbage' 'individual'
}

In [2]:
##GET TITLE/LINK/AUTHORS OF N*X ARTICLES

#loop through pages for n pages
n = 10

from tqdm.notebook import tqdm

#store results
results = pd.DataFrame(columns = ['titles', 'links', 'authors'])

for i in tqdm(range(n)):
    params['start'] = i*10
    response = requests.get(url, headers=headers, params=params)
    soup = BeautifulSoup(response.text, 'html.parser')
    articles = soup.find_all('div', class_='gs_ri')

    #get titles
    titles = [article.find('h3', class_='gs_rt').text for article in articles]
    #locate and strip the '[HTML]' or '[PDF]' in the titles
    titles = [title[:title.find('[')-1] if title.find('[') != -1 else title for title in titles]

    #get links
    links = [article.find('h3', class_='gs_rt').find('a')['href'] for article in articles]
    #get authors
    authors = [article.find('div', class_='gs_a').text for article in articles]
    
    #add to results
    results = results._append(pd.DataFrame({'titles' : titles, 'links' : links, 'authors' : authors}))

  0%|          | 0/10 [00:00<?, ?it/s]

In [3]:
print(results.head)

<bound method NDFrame.head of                                                titles  \
0   [HTML][HTML] Consumers in a circular economy: ...   
1   [HTML][HTML] Circular economy and household e-...   
2   [HTML][HTML] Household organic waste: Integrat...   
3   [HTML][HTML] Waste prevention, energy recovery...   
4   [HTML][HTML] Potential for circular economy in...   
..                                                ...   
5   Towards a circular economy: A case study of wa...   
6   Assessing the Outcomes of Circular Economy and...   
7   [PDF][PDF] Effective solid waste management in...   
8   [HTML][HTML] Co-production in solid waste mana...   
9   [HTML][HTML] Waste treatment company decision-...   

                                                links  \
0   https://www.sciencedirect.com/science/article/...   
1   https://www.sciencedirect.com/science/article/...   
2   https://www.sciencedirect.com/science/article/...   
3   https://www.sciencedirect.com/science/article/...   


In [4]:
results.to_csv('info_articles.csv')

In [5]:
# FUNCTION EXCTRACT DOIS
import re
def extract_doi_from_webpage(url):
    try:
        response = requests.get(url)
        
        if response.status_code == 200:
            soup = BeautifulSoup(response.content, 'html.parser')
            doi_elements = soup.find_all(string=re.compile(r'\b(10\.\d{4,}(?:\.\d+)*/\S+)\b'))
            
            if doi_elements:
                dois = []
                for element in doi_elements:
                    match = re.findall(r'\b(10\.\d{4,}(?:\.\d+)*/\S+)\b', str(element))
                    if match:
                        dois.extend(match)
                
                return dois
            else:
                return None
        else:
            print(f"Failed to fetch content from {url}")
            return None
    except requests.exceptions.RequestException as e:
        print(f"Error occurred while fetching content from {url}: {e}")
        return None

In [4]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
from tqdm.notebook import tqdm

# extract doi's from url
def extract_doi_from_webpage(url):
    try:
        response = requests.get(url)
        
        if response.status_code == 200:
            soup = BeautifulSoup(response.content, 'html.parser')
            doi_elements = soup.find_all(string=re.compile(r'\b(10\.\d{4,}(?:\.\d+)*/\S+)\b'))
            
            if doi_elements:
                dois = []
                for element in doi_elements:
                    match = re.findall(r'\b(10\.\d{4,}(?:\.\d+)*/\S+)\b', str(element))
                    if match:
                        #added 'https://doi.org/' in front so that iit can be callable
                        dois.extend([f"https://doi.org/{doi}" for doi in match])
                
                return dois 
            else:
                return None
        else:
            return None
    except requests.exceptions.RequestException as e:
        return None

# loop through pages for n pages
n = 10

results = pd.DataFrame(columns=['titles', 'links', 'authors', 'dois'])

for i in tqdm(range(n)):
    params['start'] = i * 10
    response = requests.get(url, headers=headers, params=params)
    soup = BeautifulSoup(response.text, 'html.parser')
    articles = soup.find_all('div', class_='gs_ri')

    # get titles
    titles = [article.find('h3', class_='gs_rt').text for article in articles]
    titles = [title[:title.find('[') - 1] if title.find('[') != -1 else title for title in titles]
    #get links
    links = [article.find('h3', class_='gs_rt').find('a')['href'] for article in articles]
    #get authors
    authors = [article.find('div', class_='gs_a').text for article in articles]

    # get doi, if it is None, keep link ^ as url of article 
    dois = [extract_doi_from_webpage(link) for link in links]
    dois_links = [doi[0] if doi else None for doi in dois]
    final_links = [doi if doi else link for doi, link in zip(dois_links, links)]

    
    results = pd.concat([results, pd.DataFrame({'titles': titles, 'links': final_links, 'authors': authors, 'dois': dois_links})])

results.to_csv('article_info3.csv', index=False)

  0%|          | 0/10 [00:00<?, ?it/s]

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


In [9]:
print(results.head)

<bound method NDFrame.head of                                                titles  \
0   [HTML][HTML] Consumers in a circular economy: ...   
1   [HTML][HTML] Circular economy and household e-...   
2   [HTML][HTML] Household organic waste: Integrat...   
3   [HTML][HTML] Waste prevention, energy recovery...   
4   [HTML][HTML] Potential for circular economy in...   
..                                                ...   
5   Towards a circular economy: A case study of wa...   
6   Assessing the Outcomes of Circular Economy and...   
7   [PDF][PDF] Effective solid waste management in...   
8   [HTML][HTML] Co-production in solid waste mana...   
9   [HTML][HTML] Waste treatment company decision-...   

                                                links  \
0   https://www.sciencedirect.com/science/article/...   
1   https://www.sciencedirect.com/science/article/...   
2   https://www.sciencedirect.com/science/article/...   
3   https://www.sciencedirect.com/science/article/...   


In [10]:
# FUNCTION TO DOWNLOAD AS PDFS USING DOIS
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time
def download_pdfs(dois, file_name):
    driver = webdriver.Chrome()  # Initialize web-finder
    main_page_url = 'https://sci-hub.ru/'  # Open sci-hub

    download_links = []  # List to store download links
    
    for doi in dois:
        try:
            driver.get(main_page_url)
            time.sleep(1)
            
            # Find search bar -> input DOI
            search_bar = driver.find_element(by=By.ID, value='request')
            search_bar.send_keys(doi)
            search_bar.send_keys(Keys.RETURN)  # To submit the search query
            time.sleep(1)
            
            # Find the download button
            download_button = driver.find_element(by=By.XPATH, value="//button[@onclick]")
            onclick_attribute = download_button.get_attribute('onclick')
            pdf_url = onclick_attribute.split("'")[1]
            download_link = 'https://sci-hub.ru' + pdf_url
            download_links.append(download_link)
            
        except Exception as e:
            print(f"An error occurred while downloading from Sci-Hub: {e}")
    
        driver.quit()
        return download_links
    # Extract URL from the onclick attribute
    # Assuming the format is location.href='URL'
    pdf_url = onclick_attribute.split("'")[1]
    download_link = 'https://sci-hub.ru' + pdf_url
    # Download the PDF using requests
    response = requests.get(download_link)
    with open(f'pdfs/{file_name}.pdf', 'wb') as file:
        file.write(response.content)
    
    driver.quit()

    return None

In [12]:
#LOOP THROUGH URLS
import os
data = pd.read_csv('article_info3.csv')
list_error_links = []  # Define an empty list to store error links if encountered

# Function to check if a file exists at a given path
def file_exists(file_path):
    return os.path.exists(file_path)

# Loop through each row in the DataFrame 'data'
for index, row in data.iterrows():
    pdf_file_path = f'pdfs/{index}.pdf'  # Define the path for the PDF file
    if file_exists(pdf_file_path):
        print("PDF already downloaded")
    else:
        if row['links'] in list_error_links:
            print('Error already encountered')
        else:
            url_returned = download_pdfs(row['links'], file_name='pdfs')  
            if url_returned is not None:
                list_error_links.append(url_returned)
                print(f'Error encountered for {url_returned}')

PDF already downloaded
An error occurred while downloading from Sci-Hub: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//button[@onclick]"}
  (Session info: chrome=120.0.6099.71); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF772A24D02+56194]
	(No symbol) [0x00007FF7729904B2]
	(No symbol) [0x00007FF7728376AA]
	(No symbol) [0x00007FF7728816D0]
	(No symbol) [0x00007FF7728817EC]
	(No symbol) [0x00007FF7728C4D77]
	(No symbol) [0x00007FF7728A5EBF]
	(No symbol) [0x00007FF7728C2786]
	(No symbol) [0x00007FF7728A5C23]
	(No symbol) [0x00007FF772874A45]
	(No symbol) [0x00007FF772875AD4]
	GetHandleVerifier [0x00007FF772D9D5BB+3695675]
	GetHandleVerifier [0x00007FF772DF6197+4059159]
	GetHandleVerifier [0x00007FF772DEDF63+4025827]
	GetHandleVerifier [0x00007FF772ABF029+687785]
	(No symbol) [0x00007FF77299B508]
	(No symbol) [0x

KeyboardInterrupt: 